In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize
import os

Questao 1

In [ ]:
class PolynomialFitter:
    def __init__(self, degree):
        self.degree = degree
        self.coefficients = None

    def fit(self, x, y):
        A = np.vander(x, self.degree + 1)
        self.coefficients = np.linalg.lstsq(A, y, rcond=None)[0]

    def predict(self, x):
        if self.coefficients is None:
            raise ValueError("O polinômio ainda não foi ajustado. Execute o método fit primeiro.")
        return np.polyval(self.coefficients, x)

    def plot(self, x, y):
        if self.coefficients is None:
            raise ValueError("O polinômio ainda não foi ajustado. Execute o método fit primeiro.")
        
        plt.scatter(x, y, label="Dados", color="blue")
        
        x_fit = np.linspace(min(x), max(x), 500)
        y_fit = self.predict(x_fit)
        
        plt.plot(x_fit, y_fit, label=f"Polinômio grau {self.degree}", color="red")
        plt.legend()
        plt.xlabel("x")
        plt.ylabel("y")
        plt.title("Ajuste de polinômio")
        plt.grid(True)
        plt.show()


if __name__ == "__main__":
    x_points = np.array([0, 1, 2, 3, 4, 5])
    y_points = np.array([1, 2, 1.5, 3.5, 2, 5])

    degree = int(input("Digite o grau "))

    fitter = PolynomialFitter(degree)
    fitter.fit(x_points, y_points)
    fitter.plot(x_points, y_points)


Questao 2

In [ ]:
class PolynomialDegreeSelector:
    def __init__(self, max_degree):
        self.max_degree = max_degree
        self.best_degree = None
        self.errors = []

    def find_best_degree(self, x, y):
        
        # Dividimos os dados em conjunto de treinamento e validação (30% - 70%)
        x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=42)
        
        self.errors = []
        
        for degree in range(1, self.max_degree + 1):
            A_train = np.vander(x_train, degree + 1)
            coefficients = np.linalg.lstsq(A_train, y_train, rcond=None)[0]
            
            A_val = np.vander(x_val, degree + 1)
            y_pred = A_val @ coefficients
            
            # Calcular o erro no conjunto de validação
            mse = mean_squared_error(y_val, y_pred)
            self.errors.append(mse)
        
        self.best_degree = np.argmin(self.errors) + 1
        return self.best_degree

    def plot_errors(self):
        if not self.errors:
            raise ValueError("Nenhum erro calculado. Execute `find_best_degree` primeiro.")
        
        plt.plot(range(1, self.max_degree + 1), self.errors, marker='o')
        plt.xlabel("Grau do Polinômio")
        plt.ylabel("Erro Quadrático Médio (MSE)")
        plt.title("Erro de Validação vs. Grau do Polinômio")
        plt.grid(True)
        plt.show()


if __name__ == "__main__":
    np.random.seed(42)
    x = np.linspace(0, 10, 100)
    y = 3 * x**3 - 2 * x**2 + x + np.random.normal(scale=100, size=x.shape)

    max_degree = 10

    selector = PolynomialDegreeSelector(max_degree)
    best_degree = selector.find_best_degree(x, y)
    print(f"Melhor grau de polinômio encontrado: {best_degree}")

    selector.plot_errors()

    fitter = PolynomialFitter(best_degree)
    fitter.fit(x, y)
    fitter.plot(x, y)


Questao 3 - item a)

In [ ]:
def objective(params, x, y):
    a, b = params
    return np.sum(np.abs(a * x + b - y))

def fit_linear_function(x, y):
    initial_guess = [0, 0]
    
    result = minimize(objective, initial_guess, args=(x, y), method='BFGS')
    
    if result.success:
        return result.x  # Retorna os coeficientes a e b
    else:
        raise ValueError("A otimização falhou.")

def plot_fit(x, y, a, b):
    plt.scatter(x, y, label="Dados", color="blue")
    x_fit = np.linspace(min(x), max(x), 100)
    y_fit = a * x_fit + b
    plt.plot(x_fit, y_fit, label=f"y = {a:.2f}x + {b:.2f}", color="red")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.title("Ajuste Linear por Minimização do Erro Absoluto")
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    x = np.array([1, 2, 3, 4, 5])
    y = np.array([2, 4, 5, 4, 5])

    a, b = fit_linear_function(x, y)
    print(f"Coeficientes ajustados: a = {a:.4f}, b = {b:.4f}")

    plot_fit(x, y, a, b)


Questao 3 - item b)

In [ ]:
def generate_points(m):
    np.random.seed(1)
    a = 6
    b = -3
    x = np.linspace(0, 10, m)
    y = a * x + b + np.random.standard_cauchy(size=m)
    return (x, y)

def save_points(points, path='test_points.txt'):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, 'wt') as f:
        for x, y in zip(points[0], points[1]):
            f.write(f'{x} {y}\n')

def objective(params, x, y):
    a, b = params
    return np.sum(np.abs(a * x + b - y))

def fit_linear_function(x, y):
    initial_guess = [0, 0]
    result = minimize(objective, initial_guess, args=(x, y), method='BFGS')
    if result.success:
        return result.x  # Retorna os coeficientes a e b
    else:
        raise ValueError("A otimização falhou.")

def process_points(m_values, output_dir='CodigosExcercicios/Resultados'):
    os.makedirs(output_dir, exist_ok=True)
    
    for m in m_values:
        points = generate_points(m)
        
        a, b = fit_linear_function(points[0], points[1])
        
        file_path = f'{output_dir}/points_{m}.txt'
        save_points(points, file_path)
        
        result_path = f'{output_dir}/result_{m}.txt'
        with open(result_path, 'wt') as f:
            f.write(f'Coeficientes ajustados para m = {m}:\n')
            f.write(f'a = {a:.4f}, b = {b:.4f}\n')

if __name__ == "__main__":
    m_values = [64, 128, 256, 512, 1024]
    process_points(m_values)


Questao 3 - item c)

In [ ]:
def objective_squared(params, x, y):
    a, b = params
    return np.sum((a * x + b - y) ** 2)

def fit_linear_function_squared(x, y):
    initial_guess = [0, 0]
    
    result = minimize(objective_squared, initial_guess, args=(x, y), method='BFGS')
    
    if result.success:
        return result.x  # Retorna os coeficientes a e b
    else:
        raise ValueError("A otimização falhou.")

def plot_fit(x, y, a, b):
    plt.scatter(x, y, label="Dados", color="blue")
    x_fit = np.linspace(min(x), max(x), 100)
    y_fit = a * x_fit + b
    plt.plot(x_fit, y_fit, label=f"y = {a:.2f}x + {b:.2f}", color="red")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.title("Ajuste Linear por Minimização do Erro Quadrático")
    plt.grid(True)
    plt.show()


if __name__ == "__main__":
    x = np.array([1, 2, 3, 4, 5])
    y = np.array([2, 4, 5, 4, 5])

    a, b = fit_linear_function_squared(x, y)
    print(f"Coeficientes ajustados: a = {a:.4f}, b = {b:.4f}")

    plot_fit(x, y, a, b)


Questao 3 - item d)
Os plots ja foram feitos ao longo dos itens acima.

Questao 3 - item e)

O metodo do Erro Absoluto e mais robusto contra outliers, ideal para dados com valores atípicos que podem distorcer a análise. Ja o do erro quadrático penaliza erros maiores de forma mais forte, sendo mais eficaz quando os erros grandes não devem ser ignorados e com boa propriedade matemática para otimização.